In [45]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import sklearn
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
import nltk
from textblob import TextBlob
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.metrics import pairwise_distances
import tensorflow_hub as hub
import tensorflow as tf
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [2]:
!pip3 install tensorflow==2.0.0
print(tf.__version__)

2.0.0


In [0]:
# ### Description for train data
# desc_files = len(os.listdir('../descriptions_train'))
# all_desc_train = []

# for i in range(desc_files):
#     empty_str = ''
#     for line in open(f'../descriptions_train/{i}.txt'):
#         empty_str += line.replace('\n',' ')
#     all_desc_train.append(empty_str)

In [0]:
# ### Tags for train data
# tag_files = len(os.listdir('../tags_train'))
# all_tags_train = []

# for i in range(tag_files):
#     nouns = ''
#     for line in open(f'../tags_train/{i}.txt'):
#         nouns += line.replace(':',' ')
#     all_tags_train.append(nouns.replace('\n', ' '))

In [0]:
# ### Description for test data
# desc_files = len(os.listdir('../descriptions_test'))
# all_desc_test = []

# for i in range(desc_files):
#     empty_str = ''
#     for line in open(f'../descriptions_test/{i}.txt'):
#         empty_str += line.replace('\n',' ')
#     all_desc_test.append(empty_str)

In [0]:
# ### Tags for test data
# tag_files = len(os.listdir('../tags_test'))
# all_tags_test = []

# for i in range(tag_files):
#     nouns = ''
#     for line in open(f'../tags_test/{i}.txt'):
#         nouns += line.replace(':',' ')
#     all_tags_test.append(nouns.replace('\n', ' '))

In [0]:
# np.save("all_desc_train", all_desc_train)
# np.save("all_tags_train", all_tags_train)
# np.save("all_desc_test", all_desc_test)
# np.save("all_tags_test", all_tags_test)

In [3]:
from google.colab import drive
drive.mount('/content/drive')
base_path = "drive/My Drive/Cornell/AML/Final"

all_desc_train = np.load(base_path+"/data/all_desc_train.npy")
all_tags_train = np.load(base_path+"/data/all_tags_train.npy")
all_desc_test = np.load(base_path+"/data/all_desc_test.npy")
all_tags_test = np.load(base_path+"/data/all_tags_test.npy")

train_1000 = pd.read_csv(base_path+'/data/features_train/features_resnet1000_train.csv', header=None)
train_2048 = pd.read_csv(base_path+'/data/features_train/features_resnet1000intermediate_train.csv', header=None)
test_1000 = pd.read_csv(base_path+'/data/features_test/features_resnet1000_test.csv', header=None)
test_2048 = pd.read_csv(base_path+'/data/features_test/features_resnet1000intermediate_test.csv', header=None)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
all_docs = []
all_docs.extend(all_desc_train)
all_docs.extend(all_desc_test)
all_docs.extend(all_tags_train)
all_docs.extend(all_tags_test)

all_docs_train = list(all_desc_train)+list(all_tags_train)

In [0]:
# train_1000 = pd.read_csv('../features_train/features_resnet1000_train.csv', header=None)
# train_2048 = pd.read_csv('../features_train/features_resnet1000intermediate_train.csv', header=None)
# test_1000 = pd.read_csv('../features_test/features_resnet1000_test.csv', header=None)
# test_2048 = pd.read_csv('../features_test/features_resnet1000intermediate_test.csv', header=None)

In [0]:
def get_num(string):
    string = string.replace('.', ' ').replace('/', ' ')
    num = [int(s) for s in string.split() if s.isdigit()]
    return num[0]

def parse_to_numpy(pd):
    images_idx = []
    for string in pd[0]:
        images_idx.append(get_num(string))

    pd.insert(1, "Image_Index", images_idx, True)
    pd = pd.sort_values(by=['Image_Index'])
    pd = pd.reset_index(drop=True)
    del pd['Image_Index']
    del pd[0]
    np = pd.to_numpy()
    return np

In [0]:
train_1000 = parse_to_numpy(train_1000)
train_2048 = parse_to_numpy(train_2048)
test_1000 = parse_to_numpy(test_1000)
test_2048 = parse_to_numpy(test_2048)

In [0]:
### Google embedding
#tf.enable_eager_execution()
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
### Overwrite old train_desc and test_desc
train_desc = embed(all_desc_train).numpy()
test_desc = embed(all_desc_test).numpy()
train_tags = embed(all_tags_train).numpy()
test_tags = embed(all_tags_test).numpy()

In [0]:
def noun_phrases(text):
  blob = TextBlob(text)
  nouns = [noun[0] for noun in filter(lambda x:x[1] in ['NN', "NNP", "NNS", "NNPS"], blob.tags)]
  noun_phrases = blob.noun_phrases

  return set(nouns+noun_phrases)

def nouns_only(docs):
    mapped_docs = []

    for doc in docs:
      nouns = noun_phrases(doc)

      noun_only_doc = [word for word in doc.split(" ") if word in nouns]
      mapped_docs.append(" ".join(set(noun_only_doc)))

    return mapped_docs

In [10]:
nouns_only([
    "Pablo is a person who works on AML with teacher and student",
    "the quick brown fox jumps over the lazy dogs",
    "the sky is blue and the sea is green, and birds fly high up in the sky"
])

['Pablo teacher student person AML', 'brown dogs fox', 'sky birds sea']

In [0]:
bagw_vectorizer = TfidfVectorizer(stop_words="english")
bagw_vectorizer.fit(all_docs_train)

train_desc_bagw = bagw_vectorizer.transform(all_desc_train).toarray()
test_desc_bagw = bagw_vectorizer.transform(all_desc_test).toarray()
train_tags_bagw = bagw_vectorizer.transform(all_tags_train).toarray()
test_tags_bagw = bagw_vectorizer.transform(all_tags_test).toarray()

In [0]:
bagw_pca = PCA(n_components=2000)
bagw_pca.fit(np.concatenate((train_desc_bagw, train_tags_bagw), axis=0))

train_desc_bagw_pca = bagw_pca.transform(train_desc_bagw)
test_desc_bagw_pca = bagw_pca.transform(test_desc_bagw)
train_tags_bagw_pca = bagw_pca.transform(train_tags_bagw)
test_tags_bagw_pca = bagw_pca.transform(test_tags_bagw)

In [0]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

d2v_vectorizer = Doc2Vec([TaggedDocument(doc.split(), [i]) for i, doc in enumerate(all_docs)],
                          vector_size=2000, window=2, min_count=1, workers=4)

In [0]:
train_desc_w2v = [d2v_vectorizer.infer_vector(sentence) for sentence in all_desc_train]
test_desc_w2v = [d2v_vectorizer.infer_vector(sentence) for sentence in all_desc_test]
train_tags_w2v = [d2v_vectorizer.infer_vector(sentence) for sentence in all_tags_train]
test_tags_w2v = [d2v_vectorizer.infer_vector(sentence) for sentence in all_tags_test]

In [0]:
train_desc_embed = embed(all_desc_train).numpy()
test_desc_emb = embed(all_desc_test).numpy()

In [0]:
# train_pic = np.hstack((train_1000, train_2048, train_tags))
# test_pic = np.hstack((test_1000, test_2048, test_tags))

train_desc = np.hstack((train_desc_embed, train_desc_bagw))
test_desc = np.hstack((test_desc_emb, test_desc_bagw))

train_pic = np.hstack((train_1000, train_tags, train_tags_bagw))
test_pic = np.hstack((test_1000, test_tags, test_tags_bagw))

In [0]:
# word2vec
# path = get_tmpfile("word2vec.model")
# model = Word2Vec(common_texts, size=100, window=5, min_count=1, workers=4)
# model.save("word2vec.model")
# model = Word2Vec.load("word2vec.model")
# model.train([all_desc_train], total_examples=1, epochs=1)

In [0]:
train_desc.shape, train_pic.shape, test_desc.shape, test_pic.shape

In [0]:
def weighted_similarity(v1, v2):
  sent_sim = cosine_similarity([v1[:512]], [v2[:512]])[0][0]
  bagw_sim = cosine_similarity([v2[512:]], [v2[512:]])[0][0]
  return 0.1*sent_sim + 0.9*bagw_sim

In [0]:
def get_prediction(vecs, pics):
    dists = pairwise_distances(vecs,pics,metric='cosine')
    return dists.T.argsort(1)

def map_20_bad(ranks):
    return np.mean([(20-rank)/20 if rank<20 else 0 for rank in ranks])

def map_20(ranks):
    return np.mean([1/(1+rank) if rank<20 else 0 for rank in ranks])

def evaluate(vectors,label_vectors):
    preds = get_prediction(vectors,label_vectors)
    ranks = [np.argwhere(vec==i)[0][0] for i,vec in enumerate(preds)]
    return np.mean(ranks),map_20_bad(ranks), map_20(ranks)

def get_top_20(descr_id):
    return preds[descr_id][:20]

def save_submission():
    data = []
    for i in range(2000):
        data.append(['%d.txt' % (i,),' '.join('%d.jpg' % (pic_id,) for pic_id in get_top_20(i))])
    pd.DataFrame(data,columns=['Descritpion_ID','Top_20_Image_IDs']).to_csv(base_path+'/submission.csv',index=False)

## Ridge

In [0]:
from sklearn.linear_model import RidgeCV

In [0]:
kf = KFold(n_splits=5)
rcv = RidgeCV(alphas=np.linspace(8,30,20))
for train_index, test_index in kf.split(train_pic):
    rcv.fit(train_pic[train_index], train_desc[train_index])
    pred = rcv.predict(train_pic[test_index])
    output = evaluate(pred, train_desc[test_index])
    print(output)

In [52]:
rcv.alpha_

16.105263157894736

In [0]:
rcv.fit(train_pic, train_desc)
prediction = rcv.predict(test_pic)
preds = get_prediction(prediction, test_desc)
save_submission()

## Forest

In [0]:
kf = KFold(n_splits=5)

forest = RandomForestRegressor()

for train_index, test_index in kf.split(train_pic):
    forest.fit(train_pic[train_index], train_desc[train_index])
    pred = forest.predict(train_pic[test_index])
    output = evaluate(pred, train_desc[test_index])
    print(output)

## NN

In [0]:
class KerasCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    if epoch % 1 == 0:
      print("\n Partial evaluation: ")
      print(evaluate(self.model.predict(self.pics), self.descr))
      print("\n")

    if epoch % 50 == 0 and epoch != 0:
      self.model.save(f'{base_path}/nn_checkpoints/{fold}_{epoch}.h5')

In [0]:
def custom_loss(y_true, y_pred):
  if y_true.shape[0] is None or y_pred.shape[0]:
    return 0.0

  s = y_pred.shape[0]
  c = 2*s**2-2*s
  cos = tf.keras.losses.CosineSimilarity(axis=1)
  res = 1 - cos(y_pred[None,:,:], y_true[:,None,:])

  ret = (c - (tf.reduce_sum(res) - s*tf.reduce_sum(tf.linalg.diag_part(res))))/s**2
  return ret

In [0]:
kf = KFold(n_splits=5)

fold = 0
for train_index, test_index in kf.split(train_pic):
    nn = tf.keras.models.Sequential()

    reg = None
    hidden_neurons = 4000
    
    nn.add(tf.keras.layers.Dense((train_pic.shape[1]+train_desc.shape[1])/2, input_dim=train_pic.shape[1], activation='relu', kernel_regularizer=reg))
    # nn.add(tf.keras.layers.Dense(hidden_neurons, activation='relu', kernel_regularizer=reg))
    # nn.add(tf.keras.layers.Dense(hidden_neurons, activation='relu', kernel_regularizer=reg))
    # nn.add(tf.keras.layers.Dropout(0.2))
    # nn.add(tf.keras.layers.Dense(hidden_neurons, activation='relu', kernel_regularizer=reg))
    # nn.add(tf.keras.layers.Dense(hidden_neurons, activation='relu', kernel_regularizer=reg))
    # nn.add(tf.keras.layers.Dense(hidden_neurons, activation='relu', kernel_regularizer=reg))
    # nn.add(tf.keras.layers.Dropout(0.2))
    nn.add(tf.keras.layers.Dense(hidden_neurons, activation='relu', kernel_regularizer=reg))

    nn.add(tf.keras.layers.Dense(train_desc.shape[1], activation='linear'))

    optim = tf.keras.optimizers.Adam(learning_rate=0.00001, epsilon=1e-07)

    nn.compile(loss="cosine_similarity", optimizer=optim, metrics=[])
    nn.summary()
    
    callback = KerasCallback()
    callback.pics = train_pic[test_index]
    callback.descr = train_desc[test_index]
    callback.fold = fold

    nn.fit(train_pic[train_index], train_desc[train_index], epochs=512, batch_size=32, callbacks=[callback])

    pred = nn.predict(train_pic[test_index])
    output = evaluate(pred, train_desc[test_index])
    print(output)
    fold += 1

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_91 (Dense)             (None, 4012)              18106156  
_________________________________________________________________
dense_92 (Dense)             (None, 4000)              16052000  
_________________________________________________________________
dense_93 (Dense)             (None, 3512)              14051512  
Total params: 48,209,668
Trainable params: 48,209,668
Non-trainable params: 0
_________________________________________________________________
Train on 8000 samples
Epoch 1/512
7968/8000 [============================>.] - ETA: 0s - loss: -0.3485
 Partial evaluation: 
(64.192, 0.31122500000000003, 0.13730977627893262)


8000/8000 [==============================] - 22s 3ms/sample - loss: -0.3488
Epoch 2/512
7968/8000 [============================>.] - ETA: 0s - loss: -0.4646
 Partial evaluation: 
(39.4235, 0.40

## XGBoost

In [0]:
import xgboost as xgb

In [0]:
bst = xgb.XGBRegressor()

In [0]:
bst.fit(train_pic, train_desc)